https://powershell.one/powershell-internals/scriptblocks/support-pipeline

In [51]:
function Count-Object
{
	begin
	{
		$x = 0
	}
	process
	{
		$x++
	}
	end
	{
		 $x
	}
}
# open 2 notepad for testing
gps notepad | Count-Object

2


In [52]:
& {
	# every expression result goto output
	2
	$null = 3 # eat the expression result
	$x = 0 # assignment return no value, so no output
	$x++ # is $x = $x + 1, is assignment so no output
	get-date -format filedate # get-date has return value, so goto output
	$x
}

2
20240205
1


In [53]:
function Count-Object
{
	begin
	{
		$x = 0
		$x # 0
		return $x # 0
	}
	process
	{
		$x++
		return $x # 1, 2
	}
	end
	{
		# every line in the end goto output
		 5 # 5
		 $x # 2
		 return $x # 2
	}
}
# open 2 notepad for testing
gps notepad | Count-Object

0
0
1
2
5
2
2


In [60]:
function Count-Object
{
	begin
	{
		$x = 0
	}
	process
	{
		$x++
		write-host "output: $x"
		return $x # 1, 2
	}
	end
	{
		# every line in the end goto output
		#$x
	}
}
'a','b' | Count-Object|Count-Object

output: 1
output: 1
1
output: 2
output: 2
2


In [72]:
function Count-Object
{
	begin
	{
		$x = 0
		$x
		$x
		write-host "init first"
	}
	process
	{
		write-host "$_ ,first process $x"
		# $x++
		# write-host $x
		# return $x
	}
	end
	{
		write-host 'end first'
	}
}
function Count-Object1
{
	begin
	{
		$x = 10
		$x
		$x
		write-host "init second"
	}
	process
	{
		$x++
		write-host "$_ ,second process $x"

		# return $x
	}
	end
	{
		write-host 'end second'
	}
}
'a','b' | Count-Object|Count-Object1

10
10
init second
0 ,second process 11
0 ,second process 12
init first
a ,first process 0
b ,first process 0
end first
end second


In [66]:
function Count-Object
{
	begin
	{
		$x = 0
		write-host "init first"
	}
	process
	{
		$x++
		write-host $x
		return $x
	}
	end
	{
		write-host 'end first'
	}
}
function Count-Object1
{
	begin
	{
		$x = 10
		write-host "init second"
	}
	process
	{
		$x++
		write-host $x

		return $x
	}
	end
	{
		write-host 'end second'
	}
}
'a','b' | Count-Object|Count-Object1

init first
init second
1
11
11
2
12
12
end first
end second


		init 1, inti 2
itemA:  step 1, step 2, resultA
itemB:  step 1, step 2, resultB
		end  1, end  2 

## Array like object

In [55]:
function Out-Voice
{
  param
  (
    [Parameter(Mandatory,ValueFromPipeline)]
    # allow string arrays:
    [string[]]
    $Text,

    [ValidateRange(-10,10)]
    [int]
    $Speed = 0
  )

  # do initialization tasks
  begin
  {
    $sapi = New-Object -ComObject Sapi.SPVoice
    $sapi.Rate = $Speed
  }

  # process pipeline input
  process
  {
    # unwrap arrays manually that were assigned to the
    # parameter directly:
    foreach($element in $Text)
    {
      $null = $sapi.Speak($element)
    }
  }
}

# accepts pipeline input:
'Hello','This is a test' | Out-Voice -Speed 6

In [56]:
function Array-Test
{
  param
  (
    [Parameter(Mandatory,ValueFromPipeline)]
    # allow string arrays:
    [string[]]
    $Text
  )

  # do initialization tasks
  begin
  {
    $x = 0
  }

  # process pipeline input
  process
  {
    # unwrap arrays manually that were assigned to the
    # parameter directly:
    foreach($element in $Text)
    {
      write-host $element
    }
  }
}

# accepts pipeline input:
'Hello','This is a test' | Array-Test

Hello
This is a test


In [57]:
function Array-Test
{
  param
  (
    [Parameter(Mandatory,ValueFromPipeline)]
    # allow string arrays:
    [string[]]
    $Text
  )

  # do initialization tasks
  begin
  {
    $x = 0
  }

  # process pipeline input
  process
  {
    # unwrap arrays manually that were assigned to the
    # parameter directly:
    write-host $Text
	write-host $text.gettype()
  }
}

# accepts pipeline input:
'Hello','This is a test' | Array-Test

Hello
System.String[]
This is a test
System.String[]


In [ ]:
function Array-Test
{
  param
  (
    [Parameter(Mandatory,ValueFromPipeline)]
    # allow string arrays:
    $Text
  )

  # do initialization tasks
  begin
  {
    $x = 0
  }

  # process pipeline input
  process
  {
    # unwrap arrays manually that were assigned to the
    # parameter directly:
    write-host $Text
	write-host $text.gettype()
  }
}

# accepts pipeline input:
'Hello','This is a test' | Array-Test

In [59]:
function Array-Test
{
  param
  (
    [Parameter(Mandatory,ValueFromPipeline)]
    $Text
  )

  # do initialization tasks
  begin
  {
    $x = 0
  }

  # process pipeline input
  process
  {
    # unwrap arrays manually that were assigned to the
    # parameter directly:
    write-host $Text
	write-host $text.gettype()
  }
}
## add ()
('Hello','This is a test') | Array-Test

Hello
System.String
This is a test
System.String


> the pipeline operator access obj or array as the left most (first) object, then every step is a partial function call
> the step function accept obj or item of the array, not the full array as one-time obj

In [12]:
function Test
{
  param
  (
    [Parameter(Mandatory,ValueFromPipeline)]
    # allow string arrays:
    $Text
  )

  # do initialization tasks
  begin
  {

    $x = $Test
    write-host "begin $Text"
  }

  # process pipeline input
  process
  {
    write-host "process $Text"
    $Text
  }

  end
	{
		write-host "end $Text"
	}
}

1,2 | Test|Test
write-host "----------"
(1,2 | Test)|Test
write-host "----------"
$v =1,2 | Test
write-host '**'
$v|Test

begin 
begin 
process 1
process 1
1
process 2
process 2
2
end 2
end 2
----------
begin 
process 1
process 2
end 2
begin 
process 1
1
process 2
2
end 2
----------
begin 
process 1
process 2
end 2
**
begin 
process 1
1
process 2
2
end 2
